## Download Mnist Data

In [ ]:
!mkdir dataset
!wget https://segmind-data.s3.ap-south-1.amazonaws.com/mnist_data.zip -P dataset
!unzip dataset/mnist_data.zip -d dataset/

## Set Segmind project 

In [ ]:
from segmind import set_project
set_project("f216694c-4e6f-4956-9df9-2341b3305a46")

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import os
from glob import glob
import keras


In [ ]:
img_folder_path = 'dataset/mnist_data/'

## Load Train Data

In [ ]:
train_X = []
train_y = []

mode = 'train'
classes = os.listdir(os.path.join(img_folder_path, mode))
for cl in classes:
    files = os.listdir(os.path.join(img_folder_path, mode, cl))
    for f in files:
        image_path= os.path.join(img_folder_path, mode, cl, f)
        img=cv2.imread(image_path,0)
        train_X.append(img)
        train_y.append(cl)

In [ ]:
train_X = np.asarray(train_X)
train_y = np.asarray(train_y)

In [ ]:
print(train_X.shape)
print(train_y.shape)

## Load Test Data

In [ ]:
test_X = []
test_y = []

mode = 'test'
classes = os.listdir(os.path.join(img_folder_path, mode))
for cl in classes:
    files = os.listdir(os.path.join(img_folder_path, mode, cl))
    for f in files:
        image_path= os.path.join(img_folder_path, mode, cl, f)
        img=cv2.imread(image_path,0)
        test_X.append(img)
        test_y.append(cl)

In [ ]:
test_X = np.asarray(test_X)
test_y = np.asarray(test_y)

In [ ]:
print(test_X.shape)
print(test_y.shape)

# Display Data

In [ ]:
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

In [ ]:
r_idx = np.random.randint(len(train_X),size=9)
plt.figure(figsize=(5,5))
for i,idx in enumerate(r_idx):
    plt.subplot(3,3,i+1)
    plt.imshow(train_X[idx], cmap=plt.get_cmap('gray'))
    plt.xlabel("Label : " + train_y[idx], fontsize=12)
plt.show()

# Pre-process Data

In [ ]:
train_X = train_X.reshape((train_X.shape[0], train_X.shape[1], train_X.shape[2], 1))
test_X = test_X.reshape((test_X.shape[0], test_X.shape[1], test_X.shape[2], 1))

# convert from integers to floats
train_norm_X = train_X.astype('float32')
test_norm_X = test_X.astype('float32')
# normalize to range 0-1
train_norm_X = train_norm_X / 255.0
test_norm_X = test_norm_X / 255.0

In [ ]:
#set number of categories
num_category = 10
# convert class vectors to binary class matrices
train_y = keras.utils.to_categorical(train_y, num_category)
test_y = keras.utils.to_categorical(test_y, num_category)

In [ ]:
print(train_y.shape)
print(test_y.shape)

# Save processed Data to directory

In [ ]:
if not os.path.exists('dataset/processed_data'):
    os.mkdir('dataset/processed_data')
    
np.save('dataset/processed_data/train_X.npy', train_norm_X)
np.save('dataset/processed_data/train_y.npy', train_y)
np.save('dataset/processed_data/test_X.npy', test_norm_X)
np.save('dataset/processed_data/test_y.npy', test_y)

# Switch the instance to GPU type

In [ ]:
from segmind import set_project
set_project("f216694c-4e6f-4956-9df9-2341b3305a46")

In [ ]:
from segmind.keras import KerasCallback
keras_cb = KerasCallback()

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras import backend as k
import matplotlib.pyplot as plt
import numpy as np

### Load processed data

In [ ]:
train_X = np.load('dataset/processed_data/train_X.npy')[:,:,:,0:1]
train_y = np.load('dataset/processed_data/train_y.npy')
test_X = np.load('dataset/processed_data/test_X.npy')[:,:,:,0:1]
test_y = np.load('dataset/processed_data/test_y.npy')

In [ ]:
train_X.shape

In [ ]:
train_y.shape

### Define model

In [ ]:
#input image size 28*28
img_rows , img_cols = 28, 28
input_shape = (img_rows, img_cols,1)
num_category = 10

In [ ]:
##model building
model = Sequential()
#convolutional layer with rectified linear unit activation
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
#32 convolution filters used each of size 3x3
#again
model.add(Conv2D(64, (3, 3), activation='relu'))
#64 convolution filters used each of size 3x3
#choose the best features via pooling
model.add(MaxPooling2D(pool_size=(2, 2)))
#randomly turn neurons on and off to improve convergence
model.add(Dropout(0.25))
#flatten since too many dimensions, we only want a classification output
model.add(Flatten())
#fully connected to get all relevant data
model.add(Dense(128, activation='relu'))
#one more dropout for convergence' sake :) 
model.add(Dropout(0.5))
#output a softmax to squash the matrix into output probabilities
model.add(Dense(num_category, activation='softmax'))

In [ ]:
#Adaptive learning rate (adaDelta) is a popular form of gradient descent rivaled only by adam and adagrad
#categorical ce since we have multiple classes (10) 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint('models/best_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


In [ ]:
batch_size = 16
num_epoch = 10
#model training
model.fit(train_X, train_y,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=(test_X, test_y),
          callbacks = [keras_cb, checkpoint])

### Evaluate Model

In [ ]:
model = load_model('models/best_model.h5')

In [ ]:
score = model.evaluate(test_X, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])